In [2]:
import pandas as pd
from connections import AWS

$\textbf{Epidemiology: Pitcher Matching}$

Attempts to find the best possible non-injured match for each pitcher based on: 
- `height`
- `mass`
- `number of pitches thrown`
- `time interval`

The last is the most difficult because it requires re-computing the number of pitches thrown between dates based on each injured pitcher. A manual distance-based algorithm is used to identify matching pitchers, and the closest match is chosen.

In [3]:
# set up AWS connection
aws = AWS()
aws.connect()

[AWS]: Port 5433 is free.
[AWS]: Connected to RDS endpoint.


$\textbf{Data Loading}$

- Cohorts
- Ball Tracking Data

In [4]:
from services.ball_tracking import *

In [5]:
""" Injured Cohort """
# load cohort metadata (mass, height, and pitches prior to injury)
cohort = aws.load_s3_object('epidemiology/cohorts/injured/pitcher_info/pitchers_0825.csv')
cohort_metadata = aws.load_s3_object('epidemiology/cohorts/injured/pitcher_info/pitchers_metadata.csv')
cohort_injured = cohort_metadata.rename(columns={'mlbam_id': 'mlbamid'}).merge(cohort[['mlbamid', 'injury_date']], on='mlbamid', how='left')

# add season to injured pitchers
cohort_injured['injury_date'] = pd.to_datetime(cohort_injured['injury_date'], errors='coerce')
cohort_injured['season'] = cohort_injured['injury_date'].apply(lambda x: x.year if pd.notnull(x) else None)

# created injured flag (:= 1)
cohort_injured['injured'] = 1

# drop pitches thrown column to replace w/ model pred counts
cohort_injured.drop(columns=['pitches_prior_to_injury'], axis=1, inplace=True)

""" Non-injured Cohort """
cohort_noninjured = aws.load_s3_object('epidemiology/cohorts/noninjured/pitcher_info/pitchers_0825.csv')
cohort_noninjured.drop(columns=['pitches_thrown'], axis=1, inplace=True)

In [6]:
""" Ball Tracking Data """
# all ball tracking data w/ model predictions
    # NOTE: some duplicates exist?
raw_data = aws.load_s3_object('epidemiology/ml/datasets/full/model_application_data.csv')
model_preds = aws.load_s3_object('epidemiology/ml/datasets/preds/model_application.csv')
model_preds.drop_duplicates(subset=['pitcher', 'game_date', 'pred_peak_evt'], inplace=True)
model_preds['season'] = model_preds['game_date'].str[:4].astype(int)                                # add season to model predictions

# get pitch & outing counts
pitch_counts = model_preds.groupby(['pitcher', 'season']).size().reset_index(name='pitches_thrown')                     # count pitches per pitcher-season combo
outing_counts = model_preds.groupby(['pitcher', 'season'])['game_date'].nunique().reset_index(name='total_outings')     # count outings per pitcher-season combo 

In [7]:
# combine cohort data
cohort_matching_data = pd.concat([cohort_injured, cohort_noninjured])

# merge pitch counts, outings into cohort data
cohort_matching_data = cohort_matching_data.merge(pitch_counts.rename(columns={'pitcher': 'mlbamid'}), on=['mlbamid', 'season'], how='inner')
cohort_matching_data = cohort_matching_data.merge(outing_counts.rename(columns={'pitcher': 'mlbamid'}), on=['mlbamid', 'season'], how='inner')

$\textbf{Matching}$

Scales based on matching criteria: __height, mass, pitches thrown, pitch type usage__.
- __Note__: Pitches thrown is based on the number of pitches w/ model predictions v. the number of counted pitches
    - Sometimes there is a slight discrepancy... this does seem to preseve more matches, though, likely because of Spring Training data
- Pitches thrown are counted between the injured pitcher's first and last pitch
- Only pitchers throwing ±10% of the injured pitcher's total pitches are considered

Euclidean distance is used to compute the matching criteria, selecting the minimum for each pitcher.

In [8]:
import pickle
from sklearn.preprocessing import StandardScaler
from services.matching import compute_matching_info

In [10]:
# create scalers for each year
matching_scalers = {}
matching_cols = ['mlbamid', 'season', 'height', 'mass', 'pitches_thrown', 'total_outings']
for s in [i for i in range(2015, 2026)]:
    # filter data for the current season, fit scaler
    season_data = cohort_matching_data[cohort_matching_data['season'] == s][matching_cols]
    scaler = StandardScaler().fit(season_data[['height', 'mass', 'pitches_thrown']])
    
    # save to dictionary
    matching_scalers[s] = scaler

# save to disc
with open('storage/matching_scalers.pkl', 'wb') as f:
    pickle.dump(matching_scalers, f)

In [17]:
# update injured data w/ counts
cohort_injured_counts = cohort_injured.merge(
    pitch_counts.rename(columns={'pitcher': 'mlbamid'}), 
    on=['mlbamid', 'season'], 
    how='inner'
)
cohort_injured_counts = cohort_injured_counts.merge(
    outing_counts.rename(columns={'pitcher': 'mlbamid'}), 
    on=['mlbamid', 'season'], 
    how='inner'
)

# iterate through injured pitchers
    # check if pitcher has pitches from season of injury (if not --> pass) 
    # get all pitches in date range, check for non-injured pitchers with pitches in date range --> compute matching info
matches = []
for idx, row in cohort_injured_counts.iterrows():
    
    # extract pitcher, season
    pitcher = row['mlbamid']
    season = row['season']
    
    # get injured pitcher's ball tracking data, season(s) w/ pitches
    inj_data_season = model_preds[
        (model_preds['pitcher'].isin(list(cohort_injured['mlbamid']))) & 
        (model_preds['season'] == season)
    ].copy()
    pitcher_bt = inj_data_season[(inj_data_season['pitcher'] == pitcher) & (inj_data_season['season'] == season)].sort_values('game_date')
    seasons_pitched = get_season_from_date(pitcher_bt['game_date'])

    # skip if pitcher has no pitches in season of injury
    if season not in seasons_pitched or pitcher_bt.empty:
        continue

    else:
        # get all pitcher metadata
        inj_ht = row['height']
        inj_mass = row['mass']
        inj_pitches = pitcher_bt.shape[0]
        inj_first_pitch_date = pitcher_bt['game_date'].min()
        inj_last_pitch_date = pitcher_bt['game_date'].max()
        inj_outing_count = pitcher_bt['game_date'].nunique()                                    # for matching based on no. of outings

        # get non-injured pitcher ball tracking data from season of injury
        noninj_data_season = model_preds[
            (model_preds['pitcher'].isin(list(cohort_noninjured['mlbamid']))) & 
            (model_preds['season'] == season)
        ].copy()

        # get all pitches btw inj_first_pitch_date & inj_last_pitch_date
            # then get pitch counts for each non-injured pitcher
        noninj_data_season = noninj_data_season[
            (noninj_data_season['game_date'] >= inj_first_pitch_date) & 
            (noninj_data_season['game_date'] <= inj_last_pitch_date)
        ]

        # get pitch counts & outing counts for non-injured pitchers in date range
        noninj_pitch_counts = noninj_data_season.groupby(['pitcher', 'season']).size().reset_index(name='pitches_thrown_interval')
        noninj_outing_counts = noninj_data_season.groupby(['pitcher', 'season'])['game_date'].nunique().reset_index(name='outings_interval')
        noninj_counts = noninj_pitch_counts.merge(noninj_outing_counts, on=['pitcher', 'season'])
        
        # skip if no non-injured pitchers have pitches in date range (e.g., spring training injury)
        if noninj_counts.empty:
            continue
        
        else:
            # get eligible non-injured pitchers w/ pitches thrown during interval
                # trim to matching columns, rename to match scaler
                # NOTE: only pitchers within 20% of injured pitcher's pitches thrown are considered eligible
            eligible_noninj = cohort_noninjured.merge(noninj_counts, left_on=['mlbamid', 'season'], right_on=['pitcher', 'season'], how='inner')
            eligible_noninj = eligible_noninj[['season', 'mlbamid', 'height', 'mass', 'pitches_thrown_interval', 'outings_interval']].rename(columns={'pitches_thrown_interval': 'pitches_thrown'})

            # NOTE (above): filter eligible non-injured pitchers to those within 10% of injured pitcher's pitches thrown
                # after, join pitch counts
            eligible_noninj_filt = eligible_noninj[
                ((eligible_noninj['pitches_thrown'] >= 0.9 * inj_pitches) &
                (eligible_noninj['pitches_thrown'] <= 1.1 * inj_pitches)) & 
                (abs(eligible_noninj['outings_interval'] - inj_outing_count) <= 2)    # within 1 outing of injured pitcher
            ].reset_index(drop=True)

            # skip if no eligible non-injured pitchers
            if eligible_noninj_filt.empty:
                continue

            # compute matching criteria
            inj_pitcher_info = pd.DataFrame([{
                'season': season,
                'mlbamid': pitcher,
                'height': inj_ht,
                'mass': inj_mass,
                'pitches_thrown': inj_pitches
            }])

            # scale matching criteria
            season_scaler = matching_scalers[season]            # load scaler
            inj_scaled = inj_pitcher_info.copy()
            noninj_scaled = eligible_noninj_filt.copy()
            inj_scaled[season_scaler.feature_names_in_] = season_scaler.transform(inj_pitcher_info[season_scaler.feature_names_in_])
            noninj_scaled[season_scaler.feature_names_in_] = season_scaler.transform(eligible_noninj_filt[season_scaler.feature_names_in_])
            
            # get matching info
            n_matches = 5
            match_info = compute_matching_info(
                inj_scaled, 
                noninj_scaled, 
                season_scaler.feature_names_in_, 
                metric='euclidean', 
                n_matches=n_matches
            )

            # store differences btw matched pitchers (inj - noninj)
            if n_matches == 1:
                matched_pitcher = eligible_noninj_filt[eligible_noninj_filt['mlbamid'] == match_info['mlbamid_noninjured']]
                
                # compute differences
                match_info['ht_diff'] = inj_ht - matched_pitcher['height'].values[0]
                match_info['mass_diff'] = inj_mass - matched_pitcher['mass'].values[0]
                match_info['pitches_thrown_diff'] = inj_pitches - matched_pitcher['pitches_thrown'].values[0]
                match_info['outings_diff'] = inj_outing_count - matched_pitcher['outings_interval'].values[0]

            else:
                matched_pitchers = eligible_noninj_filt[eligible_noninj_filt['mlbamid'].isin(match_info['mlbamid_noninjured'])]
            
            # update metadata & store
            match_info['season'] = season
            matches.append(match_info)

In [20]:
pd.DataFrame(matches)

,mlbamid_injured,mlbamid_noninjured,min_distances,season
0,506433,"[448179, 596295, 663903, 450203, 543475]","[0.041192519593004295, 0.22589446228421717, 0....",2023
1,435221,"[502009, 421685, 518774, 150359, 475416]","[1.3222249282114833, 1.71380115267746, 1.86510...",2015
2,493603,"[477229, 491708, 462382, 276542, 453172]","[0.18898184493234352, 0.4739636798837133, 0.49...",2015
3,456701,"[285064, 430661, 453286, 501957]","[0.37778426752859245, 0.7256964672920329, 0.84...",2015
4,595032,"[592717, 453562, 446372, 285079, 501992]","[0.3786026054368706, 0.4722406369154507, 0.472...",2015
...,...,...,...,...
407,695549,"[543294, 701542, 656302, 656288, 622491]","[0.4370093830911451, 0.4463595668876825, 0.447...",2025
408,669203,"[681343, 680730, 682847, 592866, 605280]","[0.25913055538331514, 0.5155851798018107, 0.62...",2025
409,594902,"[680730, 656427, 592866, 656557, 605280]","[0.17866127136210896, 0.44506941269237743, 0.4...",2025
410,669854,"[681343, 680730, 694973, 682847, 592866]","[1.1118201749281136, 1.2387690003506566, 1.342...",2025


In [24]:
# create dataframe of matches, upload to S3
    # upload based on number of outings
if n_matches == 1:
    matches_full = pd.DataFrame(matches).drop_duplicates()
    aws.upload_to_s3(matches_full, 'epidemiology/cohorts/injured/pitcher_info/matches_0825.csv')
else:
    matches_full = pd.DataFrame(matches).drop_duplicates(subset=['mlbamid_injured', 'season'])
    aws.upload_to_s3(matches_full, f'epidemiology/cohorts/injured/pitcher_info/matches_{n_matches}_per_pitcher.csv')

[AWS]: Uploaded object to s3://pitch-ml/epidemiology/cohorts/injured/pitcher_info/matches_5_per_pitcher.csv


$\textbf{Close AWS Connection}$

In [22]:
aws.close()

[AWS]: Database connection closed.
[AWS]: SSH tunnel stopped.
